In [1]:
import json
import os

import numpy as np
import polars as pol
from sklearn.model_selection import train_test_split

In [2]:
df = pol.read_csv("../data/vodafone_data_short_head_long_emails.csv")

In [3]:
df.head()

Incident_No,Description,Assigned_Group,Incident_Type,Production_Category,Operational_Category,n_words,clean_text,Assigned_Group_fixed
str,str,str,str,str,str,i64,str,str
"""INC00000776689…","""CRM Applicatio…","""War Room""","""User Service R…","""Business Appli…","""Issue_Applicat…",182,"""crm applicatio…","""war room"""
"""INC00000776477…","""HR | VOICE TT …","""CM Support""","""User Service R…","""Business Appli…","""Critical Appli…",124,"""hr | voice tt …","""cm support"""
"""INC00000776473…","""Pending Active…","""L1- CRM Postpa…","""User Service R…","""Business Appli…","""Critical Appli…",173,"""pending active…","""l1-crm postpai…"
"""INC00000776402…","""AV definitions…","""L1-Wintel""","""User Service R…","""Software_Opera…","""Other_other_ot…",412,"""av definitions…","""l1-wintel"""
"""INC00000776390…","""old Doc not re…","""L2 - CRM Ops S…","""User Service R…","""Business Appli…","""Issue_Applicat…",229,"""old doc not re…","""l2-crm ops sup…"


In [4]:
label_map = {
    k: idx for idx, k in enumerate(df["Assigned_Group_fixed"].unique().to_list())
}
label_map

{'revenue assurance solution': 0,
 'cm support': 1,
 'user id mgmt': 2,
 'bi ': 3,
 'css/ess-my vodafone': 4,
 'mva-ops': 5,
 'crm_vas_cad_ocg_msg': 6,
 'l2-crm ops support': 7,
 'l2-ipops_cms support': 8,
 'iam-sarm': 9,
 'application support': 10,
 'askevo': 11,
 'service desk-fcr': 12,
 'cs-pune-metropolitan': 13,
 'recc': 14,
 'cit-cpos-cpossupport': 15,
 'ra mediation-pune-metropolitan': 16,
 'tbo-mis-pune-metropolitan': 17,
 'kms-vsearch-vault': 18,
 'eus': 19,
 'l2 dhruv support': 20,
 'enterprise crm operation support': 21,
 'dispatcher': 22,
 'siebelcrm_hyper care_l1 ops_techm': 23,
 'icb-nld': 24,
 'mobicule': 25,
 'billing operations': 26,
 'disha crm-crm-pune-metropolitan': 27,
 'sam operation ops': 28,
 'amdocs_': 29,
 'l2-simex support': 30,
 'sso-eai-reccpe': 31,
 'oracle l1 support': 32,
 'project everest bpa': 33,
 'sso-apm-wintel': 34,
 'l1-wintel': 35,
 'bss-myworld': 36,
 'ro': 37,
 'war room': 38,
 'central application-npg': 39,
 'iam provisioning': 40,
 'sso-winte

In [5]:
df = df.with_columns(
    (pol.col("Assigned_Group_fixed").map_dict(label_map)).alias("label")
)
df.head()

Incident_No,Description,Assigned_Group,Incident_Type,Production_Category,Operational_Category,n_words,clean_text,Assigned_Group_fixed,label
str,str,str,str,str,str,i64,str,str,i64
"""INC00000776689…","""CRM Applicatio…","""War Room""","""User Service R…","""Business Appli…","""Issue_Applicat…",182,"""crm applicatio…","""war room""",38
"""INC00000776477…","""HR | VOICE TT …","""CM Support""","""User Service R…","""Business Appli…","""Critical Appli…",124,"""hr | voice tt …","""cm support""",1
"""INC00000776473…","""Pending Active…","""L1- CRM Postpa…","""User Service R…","""Business Appli…","""Critical Appli…",173,"""pending active…","""l1-crm postpai…",45
"""INC00000776402…","""AV definitions…","""L1-Wintel""","""User Service R…","""Software_Opera…","""Other_other_ot…",412,"""av definitions…","""l1-wintel""",35
"""INC00000776390…","""old Doc not re…","""L2 - CRM Ops S…","""User Service R…","""Business Appli…","""Issue_Applicat…",229,"""old doc not re…","""l2-crm ops sup…",7


In [6]:
df_train, df_test = train_test_split(
    df, test_size=0.2, random_state=32, shuffle=True, stratify=df.select("label")
)

df_val, df_test = train_test_split(df_test, test_size=0.5, shuffle=False)

print(f"Shape of the training data: {df_train.shape}")
print(f"Shape of the validation data: {df_val.shape}")
print(f"Shape of the test data: {df_test.shape}")

Shape of the training data: (9072, 10)
Shape of the validation data: (1134, 10)
Shape of the test data: (1134, 10)


In [7]:
if not os.path.exists("../data/voda_idea_data_splits/"):
    os.mkdir("../data/voda_idea_data_splits/")

In [8]:
df_train.write_csv("../data/voda_idea_data_splits/train_long_emails.csv")

In [9]:
df_val.write_csv("../data/voda_idea_data_splits/val_long_emails.csv")

In [10]:
df_test.write_csv("../data/voda_idea_data_splits/test_long_emails.csv")

In [11]:
with open("../data/voda_idea_data_splits/label_map_long_emails.json", "w") as fp:
    json.dump(label_map, fp, indent=4)